In [1]:
import random as r

class Game:
    
    def __init__(self, rows, cols, mines):
        self.rows = rows
        self.cols = cols
        self.mines = mines
        self.board = [[[] for _ in range(cols)] for _ in range(rows)]
        randoms = r.sample(range(rows * cols), mines)
        for i in range(rows):
            for j in range(cols):
                self.board[i][j] = Cell(i, j)
                if i * cols + j in randoms:
                    self.board[i][j].mine = True
        mask = ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1))
        for i in range(rows):
            for j in range(cols):
                res = 0
                for di, dj in mask:
                    if i + di in range(rows) and j + dj in range(cols):
                        if self.board[i + di][j + dj].mine:
                            res += 1
                self.board[i][j].neighbours = res
        
        
    def print_board(self):
        for i in range(self.rows):
            print(*[1 if x.mine else 0 for x in self.board[i]])
            
    def print_neighbours(self):
        for i in range(self.rows):
            print(*[x.neighbours for x in self.board[i]])
            
            
class Cell:
    def __init__(self, row, col):
        self.row = row
        self.col = col
        self.mine = False
        self.open = False

In [2]:
import sys
from PyQt5.QtWidgets import QWidget, QToolTip, QPushButton, QApplication, QDesktopWidget, QLabel 
from PyQt5.QtWidgets import QMainWindow, qApp, QAction, QMessageBox, QLineEdit, QInputDialog
from PyQt5.QtCore import QCoreApplication, QLine, Qt
from PyQt5.QtGui import QFont, QIcon, QPainter, QColor, QBrush, QPen

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

class Example(QMainWindow, ):


    def __init__(self, rows, cols, mines):
        super().__init__()
        self.mines = mines
        self.right_flags = 0
        self.rows = rows
        self.cols = cols
        self.game = Game(rows, cols, self.mines)
        self.buttons = []
        self.initUI()

    def text(self, text, x, y):
        lbl = QLabel(text, self)
        lbl.move(x, y)
                
        
    def center(self, width=800, hight=800):
        self.resize(width, hight)
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())

    
    def mine_clicked(self, btn):
        modifiers = QApplication.keyboardModifiers()
        if modifiers == Qt.ShiftModifier:
            btn.setIcon(QIcon('flag.png'))
            self.mines -= 1
            self.statusBar().showMessage(f"{self.mines} mines remain")
            self.right_flags += 1
            if self.right_flags == self.mines:
                self.statusBar().showMessage(f"You win!!!")
                reply = QMessageBox.question(self, '"You win!!!', 
                "Do you want to restart?", QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
                if reply == QMessageBox.Yes:
                    self.restart()
                else:
                    qApp.quit()
        else:
            btn.setIcon(QIcon('bomb.png'))
            self.statusBar().showMessage(f"You lost!!!")
            reply = QMessageBox.question(self, '"You lost!!!', 
            "Do you want to restart?", QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
            if reply == QMessageBox.Yes:
                self.restart()
            else:
                qApp.quit()
                
                
    def change_zero(self, i, j, btn):
        mask = ((-1, 0), (0, -1), (0, 1), (1, 0))
        for di, dj in mask:
            iq, jq = i + di, j + dj
            if iq in range(self.rows) and jq in range(self.cols):
                message = self.game.board[iq][jq].neighbours
                if message == 0:
                    for k in range(len(self.buttons)): 
                        ib, jb, qb = self.buttons[k]
                        if (ib, jb) == (iq, jq):
                            del self.buttons[k]
                            qb.setIcon(QIcon(f'{message}.png'))
                            self.change_zero(ib, jb, qb)
                            break  
                else:
                    for k in range(len(self.buttons)): 
                        ib, jb, qb = self.buttons[k]
                        if (ib, jb) == (iq, jq):
                            del self.buttons[k]
                            qb.setIcon(QIcon(f'{message}.png'))
                            break
                    

                
            
    def empty_clicked(self, btn, i, j):
        modifiers = QApplication.keyboardModifiers()
        if modifiers == Qt.ShiftModifier:
            self.mines -= 1
            self.statusBar().showMessage(f"{self.mines} mines remain")
            btn.setIcon(QIcon('flag.png'))
        else:
            message = self.game.board[i][j].neighbours
            btn.setIcon(QIcon(f'{message}.png'))
            if message == 0:
                self.change_zero(i, j, btn) 
            else:
                for k in range(len(self.buttons)): 
                    ib, jb, qb = self.buttons[k]
                    if (ib, jb) == (i, j):
                        del self.buttons[k]
                        break
                
            
                
    def button(self, name, i, j, x0, y0, size=None):
        x, y = x0 + i*size, y0 + j * size
        QToolTip.setFont(QFont('SansSerif', 10))
        btn = QPushButton('', self)
        btn.resize(size, size)
        btn.move(x, y)
        if self.game.board[i][j].mine:
            btn.clicked.connect(lambda: self.mine_clicked(btn))
        else:
            btn.clicked.connect(lambda: self.empty_clicked(btn, i, j))
        self.buttons.append((i, j, btn))
        
        
    def restart(self):
        self.__init__(self.rows, self.cols, self.mines)
        
    def add_menubar(self):
        restart = QAction(QIcon('restart.png'), '&Restart game', self)
        restart.setShortcut('Ctrl+Z')
        restart.setStatusTip('Start completely new game')
        restart.triggered.connect(self.restart)
        
        change = QAction(QIcon('restart.png'), '&Change parameters and restart', self)
        change.setShortcut('Ctrl+X')
        change.setStatusTip('Change rows, cols, mines')
        change.triggered.connect(self.showDialog)        
        
        
        menubar = self.menuBar()
        Menu = menubar.addMenu('&Menu')
        Menu.addAction(restart)
        Menu.addAction(change)
        
    def showDialog(self):

        text, ok = QInputDialog.getText(self, 'Input values',
            'rows, cols, mines')

        if ok:
            rows, cols, mines = (int(q) for q in text.split(', '))
            self.mines = mines
            self.rows = rows
            self.cols = cols
            self.__init__(rows, cols, mines)
        
    def initUI(self):        
        size = 30
        x0, y0 = 10, 30
        line = 20
        self.center(x0 + self.cols * size + line,y0 + self.rows * size + line)
        self.setWindowTitle('minesweeper')
        self.setWindowIcon(QIcon('flag.png'))
        for i in range(self.cols):
            for j in range(self.rows):
                self.button(f'{i}|{j}', i, j, x0, y0, size) 
        self.statusBar().showMessage(f"{self.mines} mines remains")
        self.add_menubar()
        self.show()


if __name__ == '__main__':

    app = QApplication(sys.argv)
    ex = Example(10, 10, 10)        
    sys.exit(app.exec_())

SystemExit: 0